# Load Decryption Key

In [3]:
from view_schema import get_df_from_db

In [5]:
USERS = get_df_from_db("user/users.db")

In [6]:
USERS

,id,username,password_hash,image_1,image_2,image_3,embedding,is_authenticated
0,1,am,scrypt:32768:8:1$aizhH1tzhmAIV3Br$3e83731a9a35...,b'gAAAAABldtvP7Yvl2PwWTzwWPLzH7D8XVYNiKdxE32Y0...,b'gAAAAABldtvQuf3GHyooWRI4E6QKzDCkyIvWa44569Va...,b'gAAAAABldtvRP-jz8OtkcwdVBjQryn45QrU5bHhrLeYD...,b'gAAAAABldtvPYli0i9Wkra_pvmILObUfMmFiZqsFLeKS...,0.0
1,2,aryan,scrypt:32768:8:1$1tcnYdBxmjRGgTb2$98076e9975a9...,b'gAAAAABlduJZeZ5kUxG8Szn1rvcI_M1eue7u1dDR3PuM...,b'gAAAAABlduJaGAssAWqTPJ6n5B4uqcVrLlKu108P9-bs...,b'gAAAAABlduJbTXwH6dJaluJeBPE6sfzLy1mhy7rhMd_r...,b'gAAAAABlduJZcpiAxS7ihJ2-1DjRIng_XdOfMwcyufQa...,NaN
2,3,irma,scrypt:32768:8:1$Rl1DwTCyJpg6x8tv$699369f55d16...,b'gAAAAABlduKvPs12PMO2VMUWgHogrE0doYCWCH2Hlfyj...,b'gAAAAABlduKxK6Kf779OOU3WkwjFm2517MgwtEsPBatq...,b'gAAAAABlduKy_yXsNpyRuV-YLlHnmK7_-_JMh1oJJF1u...,b'gAAAAABlduKv9_PRJiFdZQkdJ9BacxisW-G5AQMo6cO9...,NaN
3,4,rock,scrypt:32768:8:1$GGm8Mh1nUcVhyfxd$18cf398f7683...,b'gAAAAABlduLZMG8WFt4X6X1b_QuU2i0o-cot1FygmC7T...,b'gAAAAABlduLcWEIjQSaHYSMXrYediYT80kEiUMgCQHNe...,b'gAAAAABlduLdPk0lzpTqFwCGYiHddvS2DbbTu4nJzGJ8...,b'gAAAAABlduLZDAquybI-tYD5Leyqfvh1M0GQ73dfmsGM...,NaN
4,5,sonata,scrypt:32768:8:1$2918Jeq419j8RSZW$e7e760c1e1a4...,b'gAAAAABlduRABl--3DI_gQnyZD1yTzy6yPGeTE6pBrUq...,b'gAAAAABlduRCcuoHXvT0nvjyopABM7fBkk8uulMyIp4C...,b'gAAAAABlduRD60xiy8Y_CxG-WT4LialCeNpq1xsM_m7c...,b'gAAAAABlduRAYjhkdnPoGVtXSXisUZHB3pb3emQQFh3b...,NaN
5,6,rockdad,scrypt:32768:8:1$5hU6TJAyML3CDNDG$c6217505c009...,b'gAAAAABlduSKbq6YnHFA0CuhrKJ-y4qJdPvEIHz9MEgh...,b'gAAAAABlduSM_voqXKxyjhjY2JkDCFTTYVlay6RMCE65...,b'gAAAAABlduSMN_LVP67r9OxdmkVbOthN_2e-ga9Dslpx...,b'gAAAAABlduSKKsVTCaO5yskj07wcRLYiFBVe8_Y7g8JE...,NaN
6,7,armaan,scrypt:32768:8:1$a2JcX8HK57ytq5fb$71fdaa0aaaed...,None,None,None,None,NaN
7,8,monty,scrypt:32768:8:1$WzFyXxrs0IZ5PbAp$7732902b2247...,b'gAAAAABldwqu4AVsY3o-1VX-h1GYEZ07YYYdCauu61ry...,b'gAAAAABldwqvIakPkDsWJU6jeUDTXKcLcNxO0ZegbXKs...,b'gAAAAABldwqwuiInwQbSZvUDxhhkZdmaIPSeyB4OHaEy...,b'gAAAAABldwquiEtdaa7LGf3Mv6c5vw262_RbZmgpBJKO...,NaN
8,9,monit,scrypt:32768:8:1$lT3ywLH0k0hi7BNP$a324b93e222c...,b'gAAAAABldwscnvQTBUKl3QVlePNOj_xho3GDEU1N_lyp...,b'gAAAAABldwsetI4XsFAdzc9Ba_rOF1KF8b9-Wxzixvs5...,b'gAAAAABldwsfPY-tebGKUeYrXjVgVVlnQxMT_KQoQfIO...,b'gAAAAABldwscKV1PyvHVl0GV59ycTQq1XiYlVrRFcCSY...,NaN
9,10,foo,scrypt:32768:8:1$pqvI03hzkmdlKO97$c47821e31ee9...,b'gAAAAABleCKCdYvQyhtRtwOg3zMbEAcQ4opCWDU9AWue...,b'gAAAAABleCKDSUsF6zZmBqrsbIbzO4jaXOliDtVuKhgw...,b'gAAAAABleCKD5Bv9Yqo3oSb3ePK6StYrpPbGMioZi1pT...,b'gAAAAABleCKCmlHkGLnzPukCvvgJcgait_T2a9yygLfn...,NaN
